In [13]:
from matplotlib import pyplot as plt 
%matplotlib inline
import numpy as np 

In [14]:
import csv

with open('training_descriptors/halflife_1D2Dfingerprint.csv', 'rb') as fx:
    reader = csv.reader(fx)
    raw_list = list(reader)

def float_list(some_list):
    return [float(i) for i in some_list]
    
data_X = raw_list[1:]
#Change Here to Select Features
feature_list = raw_list[0][1445:3493]
data_X = np.asarray([float_list(i[1445:3493]) for i in data_X])
print data_X.shape
    
with open('training_csv/training_halflife.csv', 'rb') as fy:
    reader = csv.reader(fy)
    raw_list = list(reader)

data_Y_reg = [i[10] for i in raw_list[1:]]


def class_to_bool(st):
    if(st == "0"):
        return 0
    elif(st == "1"):
        return 1

data_Y_cla = np.asarray([class_to_bool(i[10]) for i in raw_list[1:]])
    
    
print data_Y_cla.shape

(4362, 2048)
(4362,)


In [15]:
for i in range(len(feature_list)):
    print "feature # %d %s" % (i, feature_list[i]) 

feature # 0 FP1
feature # 1 FP2
feature # 2 FP3
feature # 3 FP4
feature # 4 FP5
feature # 5 FP6
feature # 6 FP7
feature # 7 FP8
feature # 8 FP9
feature # 9 FP10
feature # 10 FP11
feature # 11 FP12
feature # 12 FP13
feature # 13 FP14
feature # 14 FP15
feature # 15 FP16
feature # 16 FP17
feature # 17 FP18
feature # 18 FP19
feature # 19 FP20
feature # 20 FP21
feature # 21 FP22
feature # 22 FP23
feature # 23 FP24
feature # 24 FP25
feature # 25 FP26
feature # 26 FP27
feature # 27 FP28
feature # 28 FP29
feature # 29 FP30
feature # 30 FP31
feature # 31 FP32
feature # 32 FP33
feature # 33 FP34
feature # 34 FP35
feature # 35 FP36
feature # 36 FP37
feature # 37 FP38
feature # 38 FP39
feature # 39 FP40
feature # 40 FP41
feature # 41 FP42
feature # 42 FP43
feature # 43 FP44
feature # 44 FP45
feature # 45 FP46
feature # 46 FP47
feature # 47 FP48
feature # 48 FP49
feature # 49 FP50
feature # 50 FP51
feature # 51 FP52
feature # 52 FP53
feature # 53 FP54
feature # 54 FP55
feature # 55 FP56
feature # 5

In [48]:
def model_performance(y_truth, y_pred):

    TP, TN, P, N, FP, FN = [0 for i in range(6)]
    for i in range(len(y_truth)):
        if(y_truth[i] == 0):
            N += 1
            if(y_pred[i] == 0):
                TN += 1
            elif(y_pred[i] == 1):
                FN += 1
        elif(y_truth[i] == 1):
            P += 1
            if(y_pred[i] == 1):
                TP += 1
            elif(y_pred[i] == 0):
                FP += 1
            
    print "Accuracy: %0.2f" % ((TP+TN)*1.0/(TP+TN+FP+FN))
    print "Sensitivity: %0.2f" % (TP*1.0/P)
    print "Specificity: %0.2f" % (TN*1.0/N)

    

In [16]:
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit

cv = ShuffleSplit(n_splits=10, test_size=0.1)

In [17]:

#train_X, test_X, train_y, test_y = train_test_split(data_X, data_Y_cla, test_size = 0.2)

decision_tree = tree.DecisionTreeClassifier(max_depth=1024, max_features=256, min_samples_split=4, class_weight="balanced")
#decision_tree.fit(train_X, train_y)

scores = cross_val_score(decision_tree, data_X, data_Y_cla, cv=cv)

print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

#test_y_pre = decision_tree.predict(test_X)
#print(test_y_pre)
#print(test_y)

Accuracy: 0.74 (+/- 0.03)


In [49]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
#Test Performance Metrics
kf = KFold(n_splits=10, shuffle=True)
y_pred = cross_val_predict(decision_tree, data_X, data_Y_cla, cv=kf)
model_performance(data_Y_cla, y_pred)
metrics.accuracy_score(data_Y_cla, y_pred)

Accuracy: 0.74
Sensitivity: 0.78
Specificity: 0.68


0.74140302613480058

In [18]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(max_depth=512, n_estimators=32, max_features=64, class_weight="balanced", min_samples_split=8)
#random_forest.fit(train_X, train_y)

scores = cross_val_score(random_forest, data_X, data_Y_cla, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

#test_y_pre = random_forest.predict(test_X)
#print(test_y_pre)


Accuracy: 0.81 (+/- 0.03)


In [50]:
kf = KFold(n_splits=10, shuffle=True)
y_pred = cross_val_predict(random_forest, data_X, data_Y_cla, cv=kf)
model_performance(data_Y_cla, y_pred)
metrics.accuracy_score(data_Y_cla, y_pred)

Accuracy: 0.79
Sensitivity: 0.86
Specificity: 0.66


0.78862906923429621

In [19]:
from sklearn import svm
linear_svm = svm.LinearSVC(C=0.001, class_weight="balanced")
#linear_svm.fit(train_X, train_y)

scores = cross_val_score(linear_svm, data_X, data_Y_cla, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

#test_y_pre = linear_svm.predict(test_X)
#print(test_y_pre)
#print(test_y)

Accuracy: 0.77 (+/- 0.03)


In [35]:
from sklearn import svm
rbf_svm = svm.SVC(C=10, class_weight="balanced")
#linear_svm.fit(train_X, train_y)

scores = cross_val_score(rbf_svm, data_X, data_Y_cla, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

#test_y_pre = linear_svm.predict(test_X)
#print(test_y_pre)
#print(test_y)

Accuracy: 0.79 (+/- 0.05)


In [36]:
from sklearn import linear_model
logistic_reg = linear_model.LogisticRegression(class_weight="balanced", C=0.05)
scores = cross_val_score(logistic_reg, data_X, data_Y_cla, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


Accuracy: 0.77 (+/- 0.02)


In [31]:
from sklearn import linear_model
logistic_reg_cv = linear_model.LogisticRegressionCV(class_weight="balanced", cv=10)
logistic_reg_cv.fit(data_X, data_Y_cla)

#scores = cross_val_score(logistic_reg, data_X, data_Y_cla, cv=cv)
#print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

test_y_pre = logistic_reg_cv.predict(data_X)
#score = logistic_reg_cv.score(data_X, data_Y_cla)
#print("Accuracy: %0.2f" % (score))
#print(test_y_pre)


In [32]:
print logistic_reg_cv.scores_
print logistic_reg_cv.Cs_
print logistic_reg_cv.C_

{1: array([[ 0.64611872,  0.65296804,  0.65753425,  0.64383562,  0.64383562,
         0.6369863 ,  0.65068493,  0.62785388,  0.6369863 ,  0.63242009],
       [ 0.59174312,  0.64449541,  0.65825688,  0.69036697,  0.69266055,
         0.6766055 ,  0.69724771,  0.69266055,  0.69724771,  0.68807339],
       [ 0.69036697,  0.66972477,  0.67201835,  0.64449541,  0.63761468,
         0.64220183,  0.63073394,  0.61926606,  0.62844037,  0.62155963],
       [ 0.51834862,  0.52752294,  0.58256881,  0.62155963,  0.59862385,
         0.59862385,  0.58715596,  0.5940367 ,  0.58944954,  0.59174312],
       [ 0.67431193,  0.66972477,  0.69036697,  0.70412844,  0.69495413,
         0.68577982,  0.68348624,  0.68348624,  0.69266055,  0.6766055 ],
       [ 0.73394495,  0.74541284,  0.72247706,  0.72477064,  0.69266055,
         0.70412844,  0.69495413,  0.69266055,  0.69266055,  0.68348624],
       [ 0.50917431,  0.58486239,  0.60091743,  0.62614679,  0.64678899,
         0.64449541,  0.63990826,  0.6399

In [17]:
from sklearn import neighbors
KNN = neighbors.KNeighborsClassifier(10, weights="distance")
KNN.fit(train_X, train_y)
test_y_pre = KNN.predict(test_X)
print(test_y_pre)


[1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 0 0 0 0 1 1 0 1 0 1
 0 1 1 1 1 1 0 0 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 0
 0 1 0 1 0 0 1 1 1 1 1 1 0 1 1 1 0 1 0 1 0 0 1 0 0 1 1 1 0 1 1 1 1 0 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 0
 1 1 0 0 1 0 0 0 1 0 1 1 1 1 0 1 0 1 0 1 0 1 0 1 0 1 1 0 0 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 0 0 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 0 0 0 1 1 1 0 1 1 1 1 0 1
 1 1 1 0 1 1 1 0 1 0 1 0 0 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1
 0 1 1 1 0 0 1 0 1 1 1 1 1 1 0 1 0 0 0 1 0 1 0 1 1 1 0 1 1 1 1 0 0 0 0 1 1
 0 0 1 1 1 1 0 1 0 1 1 0 1 1 0 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 0
 0 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 0 1 1 1 1 1 0 0 1 1 0 0 1 0 1 1 1 1 0 0 1
 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 0 0 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1
 1 1 0 0 0 1 0 1 1 1 0 0 

In [18]:
TP, TN, P, N, FP, FN = [0 for i in range(6)]
for i in range(len(test_y)):
    if(test_y[i] == 0):
        N += 1
        if(test_y_pre[i] == 0):
            TN += 1
        elif(test_y_pre[i] == 1):
            FN += 1
    elif(test_y[i] == 1):
        P += 1
        if(test_y_pre[i] == 1):
            TP += 1
        elif(test_y_pre[i] == 0):
            FP += 1

            
print "Accuracy: %f" % ((TP+TN)*1.0/(TP+TN+FP+FN))
print "Sensitivity: %f" % (TP*1.0/P)
print "Specificity: %f" % (TN*1.0/N)

Accuracy: 0.734250
Sensitivity: 0.849091
Specificity: 0.538700
